In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from keras import Input

In [2]:
from keras.utils import load_img, img_to_array

In [6]:
with open('./data/integrated.pkl', 'rb') as f:
    df = pickle.load(f)

In [8]:
df = df[23191:25990]

In [11]:
df.iloc[[0, -1], :]

,f10-7,mag,Np,Vp,Tpr,ssn,img_193_loc,img_304_loc
2010-09-01,77.7,7.345958,None,None,None,51,./data/img_193/2010_09_01.jpg,./data/img_304/2010_09_01.jpg
2018-04-30,71.3,5.670791,None,None,None,0,./data/img_193/2018_04_30.jpg,./data/img_304/2018_04_30.jpg


In [18]:
df.head()

,f10-7,img_193_loc,img_304_loc
2010-09-01,77.7,./data/img_193/2010_09_01.jpg,./data/img_304/2010_09_01.jpg
2010-09-02,78.4,./data/img_193/2010_09_02.jpg,./data/img_304/2010_09_02.jpg
2010-09-03,78.5,./data/img_193/2010_09_03.jpg,./data/img_304/2010_09_03.jpg
2010-09-04,83.6,./data/img_193/2010_09_04.jpg,./data/img_304/2010_09_04.jpg
2010-09-05,83.5,./data/img_193/2010_09_05.jpg,./data/img_304/2010_09_05.jpg


In [14]:
df = df[['f10-7', 'img_193_loc', 'img_304_loc']]

In [22]:
def imgarr(img_loc):
    try:
        img_ = load_img(img_loc, target_size=(512, 512))
        img = img_to_array(img_)[:, :, 0].reshape(512, 512, 1)
        img = img / 255.0
    except:
        img = np.zeros((512, 512, 1))
    return img

In [23]:
def ardatagen(df):
    img193, img304, value = [], [], []
    for i in range(len(df)):
        img193.append(imgarr(df.iloc[i, 1])) # 193
        img304.append(imgarr(df.iloc[i, 2])) # 304
        value.append(df.iloc[i, 0]) # f10-7
    return np.array(img193), np.array(img304), np.array(value)

In [24]:
img193, img304, value = ardatagen(df)

In [25]:
img193.shape

(2799, 512, 512, 1)

In [27]:
for i in range(len(img193)):
    if img193[i].sum()==0: img193[i] = img193[i-1]

In [29]:
with open('./data/img193.pkl', 'wb') as f:
    pickle.dump(img193, f)

In [30]:
for i in range(len(img304)):
    if img304[i].sum()==0: img304[i] = img304[i-1]

In [31]:
with open('./data/img304.pkl', 'wb') as f:
    pickle.dump(img304, f)

In [ ]:
with open('./data/value.pkl', 'wb') as f:
    pickle.dump(value, f)

In [ ]:
def create_regression_cnn(input_shape):
    model = Sequential()
    
    # Convolutional layers
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    # Flattening the convolutional layer output
    model.add(Flatten())
    
    # Fully connected layers
    model.add(Dense(512, activation='relu'))
    
    # Output layer for regression (single neuron, linear activation)
    model.add(Dense(1, activation='linear'))
    
    return model
